In [1]:
import pandas as pd
import numpy as np

In [2]:
# create server status table
# link user engagement & retention & payment with server status
# ml
# tbh personally would be more interested in ranking, d_ranking, etc

In [3]:
S_ROOT = r'data/'

In [4]:
df_createrole_login = pd.read_csv(S_ROOT + 'createrole_login_record.csv')
df_session = pd.read_csv(S_ROOT + 'user_daily_session_length_record.csv')
df_p = pd.read_csv(S_ROOT + 'user_payment_record.csv').rename({'date':'dt'}, axis=1)

In [5]:
df_createrole_login

,appid,dt,ctwid,server_id,type,time
0,test,2024-11-17,84a787b8d4,44,g_login,1731843941000
1,test,2024-11-17,42fa4799c1,34,g_login,1731843977000
2,test,2024-11-17,96c22d3440,60,g_login,1731843979000
3,test,2024-11-17,def76d1d2b,9,g_login,1731843937000
4,test,2024-11-17,5ece9bf10e,18,g_login,1731843984000
...,...,...,...,...,...,...
3832730,test,2024-10-18,f42abc1ce5,1,g_login,1729263590000
3832731,test,2024-10-18,be64dcba91,1,g_login,1729263529000
3832732,test,2024-10-18,e7b71af6bf,1,g_login,1729263598000
3832733,test,2024-11-03,745a5a9d69,3,g_login,1730645998000


In [6]:
df_session

,appid,dt,ctwid,session_length
0,test,2024-11-17,a512b8cf9b,17
1,test,2024-11-17,5601d6307c,1898
2,test,2024-11-17,a0e52a52cc,1894
3,test,2024-11-17,88321a495c,1
4,test,2024-11-17,0d649666d6,6185
...,...,...,...,...
2755439,test,2024-10-29,8f3fd1c742,12947
2755440,test,2024-10-29,3a6d1ab888,316
2755441,test,2024-10-29,a7659fd0d9,5295
2755442,test,2024-10-29,538844702d,601


In [7]:
df_p

,appid,dt,ctwid,server_id,time,pay_amt
0,test,2024-11-13,23dccf736f,2,1731509951124,100
1,test,2024-11-13,2cac52ff60,3,1731509996850,100
2,test,2024-11-13,af687842ae,43,1731509995400,100
3,test,2024-11-14,44f3cee987,43,1731510028964,1
4,test,2024-11-14,01c6034980,9,1731510053112,79
...,...,...,...,...,...,...
106505,test,2024-11-06,67e0a2383a,4,1730904349467,100
106506,test,2024-11-06,2d019a85af,4,1730904653874,100
106507,test,2024-11-06,a9fff2307d,8,1730904862305,100
106508,test,2024-11-06,e64eb351e8,14,1730905048959,1


In [9]:
for df in (df_createrole_login, df_session, df_p):
    df['dt'] = pd.to_datetime(df['dt'])

In [12]:
df_createrole = df_createrole_login[df_createrole_login['type']=='g_createrole']
df_server_creation = df_createrole.groupby('server_id')['dt'].min().reset_index().rename({'dt':'server_creation_dt'}, axis=1)

df_server_status = df_createrole.groupby(['server_id', 'dt'])['type']\
    .count().reset_index().rename({'type':'server_new_account'}, axis=1)
df_server_status['server_total_account'] = df_server_status.groupby('server_id')['server_new_account']\
    .rolling(999, min_periods=1).sum().values
df_server_status = df_server_status.merge(df_server_creation, on='server_id')
df_server_status['server_day'] = [x.days for x in df_server_status['dt'] - df_server_status['server_creation_dt']]

df_login = df_createrole_login[df_createrole_login['type']=='g_login']
df_server_status['daily_active_account'] = df_login.groupby(['server_id', 'dt'])['type'].count().values


df_server_status

ValueError: Length of values (679) does not match length of index (153)